### 1. Configuration & Setup

In [3]:
import pandas as pd
import networkx as nx
import community.community_louvain as community_louvain 
from pathlib import Path
import matplotlib.pyplot as plt

# ==========================================
# 1. CONFIGURATION & SETUP
# ==========================================
# Exact path from your provided code
PROCESSED_DIR = Path("C:/Users/biagu/Documents/GitHub/social_graphs_project/data/processed")

NODES_PATH = PROCESSED_DIR / "nodes_enriched.csv"
EDGES_PATH = PROCESSED_DIR / "edges_enriched.csv"
OUTPUT_PATH = PROCESSED_DIR / "nodes_analyzed.csv"

print(f"Loading data from: {PROCESSED_DIR}")

# ==========================================
# 2. BUILD GRAPH
# ==========================================
nodes_df = pd.read_csv(NODES_PATH)
edges_df = pd.read_csv(EDGES_PATH)

print(f"Nodes loaded: {len(nodes_df)}")
print(f"Edges loaded: {len(edges_df)}")


Loading data from: C:\Users\biagu\Documents\GitHub\social_graphs_project\data\processed
Nodes loaded: 2410
Edges loaded: 4805


## 2. Build graph 

In [4]:
# Initialize Graph
G = nx.Graph()

# Add Nodes
for _, row in nodes_df.iterrows():
    # We treat 'node_id' as the unique identifier
    G.add_node(row['node_id'], 
               name=row['name'], 
               type=row['node_type'], 
               description=str(row['description']))

# Add Edges
for _, row in edges_df.iterrows():
    G.add_edge(row['source'], row['target'], 
               relation=row['relationship'], 
               type=row['edge_type'])

print(f"Graph constructed: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges.")

Graph constructed: 2410 nodes, 4669 edges.


### 3. Community detection (The factions)

In [5]:
# ==========================================
# 3. COMMUNITY DETECTION (The Factions)
# ==========================================
print("\n--- Detecting Communities (Louvain) ---")
# Compute partition
partition = community_louvain.best_partition(G)

# Add 'community' column to the DataFrame
nodes_df['community'] = nodes_df['node_id'].map(partition)

# Analyze Community Sizes
comm_counts = nodes_df['community'].value_counts()
print(f"Total Communities Detected: {len(comm_counts)}")
print("\nTop 10 Largest Communities:")
print(comm_counts.head(10))

# Print sample members of top communities to verify they make sense
print("\n--- Community Membership Preview ---")
for comm_id in comm_counts.head(5).index:
    members = nodes_df[nodes_df['community'] == comm_id]['name'].tolist()
    # Handle cases where names might be missing/NaN
    clean_members = [str(m) for m in members if pd.notna(m)]
    print(f"\nCommunity {comm_id} (Size: {len(members)}):")
    print(f"  Sample Members: {', '.join(clean_members[:8])}...")


--- Detecting Communities (Louvain) ---
Total Communities Detected: 542

Top 10 Largest Communities:
community
9      308
6      202
41     201
97     148
135    102
16      99
42      92
1       80
114     74
129     74
Name: count, dtype: int64

--- Community Membership Preview ---

Community 9 (Size: 308):
  Sample Members: Bloody Finger, Grave Glovewort [1], Grave Glovewort [2], Ghost Glovewort [1], Fireproof Dried Liver, Fire Grease, Throwing Dagger, Remembrance Of The Blasphemous...

Community 6 (Size: 202):
  Sample Members: Memory Of Grace, Smithing Stone [2], Ancient Dragon Smithing Stone, Somber Ancient Dragon Smithing Stone, Rowa Raisin, Pauper's Rune, Pickled Turtle Neck, Gold-pickled Fowl Foot...

Community 41 (Size: 201):
  Sample Members: Armorer's Cookbook [2], Armorer's Cookbook [3], Armorer's Cookbook [4], Armorer's Cookbook [1], Armorer's Cookbook [6], Armorer's Cookbook [7], Hero's Rune [1], Celebrant's Cleaver...

Community 97 (Size: 148):
  Sample Members: Baldac

### 4. Centrality Analysis (The "True" Lore Bridges)
Here we calculate Betweenness Centrality. Crucially, we filter out generic "Wiki Header" nodes (like "Items" or "Weapons") that distort the graph, ensuring we find the true narrative bridges.

In [6]:
# ==========================================
# 4. CENTRALITY ANALYSIS (The "True" Lore Bridges)
# ==========================================
print("\n--- Calculating Centrality (Filtered) ---")

# 1. Define Generic Wiki Headers to Filter Out
# These distort centrality because everything links to "Items" or "Weapons"
generic_headers = [
    "Items", "Weapons", "Armors", "Incantations", "Sorceries", 
    "Talismans", "Ash of War", "Spirit Ashes", "Shields", 
    "Consumables", "Key Items", "Bosses", "NPCs", "Locations",
    "Creatures", "Enemies", "Materials"
]

# 2. Create a temporary view of the graph without these nodes
G_lore = G.copy()
nodes_to_remove = [n for n, data in G_lore.nodes(data=True) if data.get('name') in generic_headers]
G_lore.remove_nodes_from(nodes_to_remove)

print(f"Temporarily removed {len(nodes_to_remove)} generic header nodes for accurate lore analysis.")

# 3. Calculate Betweenness Centrality
betweenness = nx.betweenness_centrality(G_lore)

# 4. Map back to DataFrame (fill NaN with 0 for nodes we removed)
nodes_df['centrality'] = nodes_df['node_id'].map(betweenness).fillna(0)

# 5. Show Top Characters
print("\nTop 15 Most Central Characters/Items (Lore Accurate):")
print(nodes_df[['name', 'node_type', 'centrality']]
      .sort_values('centrality', ascending=False)
      .head(15))

# ==========================================
# 5. EXPORT RESULTS
# ==========================================
# Save the dataframe with 'community' and 'centrality' added
nodes_df.to_csv(OUTPUT_PATH, index=False)
print(f"\n✅ Analysis Complete. Saved results to: {OUTPUT_PATH}")


--- Calculating Centrality (Filtered) ---
Temporarily removed 8 generic header nodes for accurate lore analysis.

Top 15 Most Central Characters/Items (Lore Accurate):
                       name node_type  centrality
2390               Strength    weapon    0.134970
1900                  Armor      boss    0.129357
2295              Dexterity    weapon    0.076896
2133  Shadow of the Erdtree      boss    0.075557
2162      The Lands Between      boss    0.058631
2153                  Stats      boss    0.036609
2121                  Runes      boss    0.030936
2154         Status Effects      boss    0.028386
2035               Limgrave      boss    0.023754
2085                 Poison      boss    0.021008
2129            Scarlet Rot      boss    0.018518
2238            Two Fingers       npc    0.017792
2037   Liurnia of the Lakes      boss    0.017696
2335           Intelligence    weapon    0.016309
1928                 Caelid      boss    0.014988

✅ Analysis Complete. Saved res